In [1]:
import os
import MDAnalysis as mda
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
u = mda.Universe('../start.pdb')

ag_lig_1 = u.select_atoms('resname EPJ').split('residue')[0]

prot_binding_site = u.select_atoms('name CA and byres (protein and around 4 group ag_lig_1)', ag_lig_1=ag_lig_1)

#sort prot_resids
chain_resids = prot_binding_site.segids[np.argsort(prot_binding_site.resids)]
prot_resids = np.sort(prot_binding_site.resids)

prot_resids = prot_resids[np.argsort(chain_resids)]
chain_resids = np.sort(chain_resids)

print(prot_resids)
print(chain_resids)

next_subunit_dict = {'A': 'C',
                     'B': 'D',
                     'C': 'E',
                     'D': 'F',
                     'E': 'G',
                     'F': 'H',
                     'G': 'I',
                     'H': 'J',
                     'I': 'A',
                     'J': 'B'}

prot_selections = []

for i in range(5):
    # only makes senses for homomers
    selection = 'protein and ('
    for j in range(prot_resids.shape[0]):
        # iterate subunit
        selection += f'(resid {prot_resids[j]} and segid {chain_resids[j]}) or '
        chain_resids[j] = next_subunit_dict[chain_resids[j]]
    selection = selection[:-4] + ')'
    prot_selections.append(selection)

for i, selection in enumerate(prot_selections):
    print(selection)
    ag = u.select_atoms(selection)
    ag.write(f'bining_sites.vmd', mode='a',name='mda_' + str(i))

# see: https://docs.mdanalysis.org/stable/documentation_pages/selections/vmd.html#MDAnalysis.selections.vmd.SelectionWriter

[100 155 156 157 197 199 200 204  57  81 109 110 111 119 121]
['A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'C' 'C' 'C' 'C' 'C' 'C' 'C']
protein and ((resid 100 and segid A) or (resid 155 and segid A) or (resid 156 and segid A) or (resid 157 and segid A) or (resid 197 and segid A) or (resid 199 and segid A) or (resid 200 and segid A) or (resid 204 and segid A) or (resid 57 and segid C) or (resid 81 and segid C) or (resid 109 and segid C) or (resid 110 and segid C) or (resid 111 and segid C) or (resid 119 and segid C) or (resid 121 and segid C))
protein and ((resid 100 and segid C) or (resid 155 and segid C) or (resid 156 and segid C) or (resid 157 and segid C) or (resid 197 and segid C) or (resid 199 and segid C) or (resid 200 and segid C) or (resid 204 and segid C) or (resid 57 and segid E) or (resid 81 and segid E) or (resid 109 and segid E) or (resid 110 and segid E) or (resid 111 and segid E) or (resid 119 and segid E) or (resid 121 and segid E))
protein and ((resid 100 and segid E) or (resid 1

# Binding site restraints

In [33]:
binding_sites = []

# only two binding sites
for selection in np.asarray(prot_selections)[[0,3]]:
    binding_sites.append(u.select_atoms(selection))

with mda.selections.gromacs.SelectionWriter('index_awh.ndx', mode='w') as ndx:
    ndx.write(u.atoms, name='SYSTEM')
    ndx.write(u.select_atoms('protein'), name='PROT')
    ndx.write(u.select_atoms('resname POPC'), name='MEMB')
    ndx.write(u.select_atoms('resname TIP3 SOD CLA EPJ'), name='SOL_ION')
    ndx.write(u.select_atoms('resname EPJ'), name='EPJ')
    for i, epj_agg in enumerate(u.select_atoms('resname EPJ').split('residue')):
        ndx.write(epj_agg, name=f'EPJ{i+1}')
        ndx.write(epj_agg.select_atoms('name N1'), name=f'EPJ{i+1}_NT')
        ndx.write(epj_agg.select_atoms('name C10'), name=f'EPJ{i+1}_CT')
    for i, binding_site in enumerate(binding_sites):
        ndx.write(binding_site, name=f'BS{i+1}')

# Loop C

In [ ]:
#LoopC resids: chain A G and resid 193-206
#LoopC resids: chain C E I and resid 185-198

In [5]:
ch_1_loopc = u.select_atoms('segid A and resid 193-206 and name CA')
ch_2_loopc = u.select_atoms('segid C and resid 185-198 and name CA')
ch_3_loopc = u.select_atoms('segid E and resid 185-198 and name CA')
ch_4_loopc = u.select_atoms('segid G and resid 193-206 and name CA')
ch_5_loopc = u.select_atoms('segid I and resid 185-198 and name CA')

loopc_ags = [ch_1_loopc, ch_2_loopc, ch_3_loopc, ch_4_loopc, ch_5_loopc]

In [6]:
with mda.selections.gromacs.SelectionWriter('index_awh.ndx', mode='a') as ndx:
    for i, loopc in enumerate(loopc_ags):
        ndx.write(loopc, name=f'LoopC{i+1}')

# Pore restraints

In [27]:
#Pore resids: chain A G and resid 247 250 254 257 261 264 268
#Pore resids: chain C E I and resid 239 242 246 249 253 256 260

In [28]:
u = mda.Universe('../start.pdb')

In [29]:
ch_1_pore = u.select_atoms('segid A and resid 247 250 254 257 261 264 268 and name CA')
ch_2_pore = u.select_atoms('segid C and resid 239 242 246 249 253 256 260 and name CA')
ch_3_pore = u.select_atoms('segid E and resid 239 242 246 249 253 256 260 and name CA')
ch_4_pore = u.select_atoms('segid G and resid 247 250 254 257 261 264 268 and name CA')
ch_5_pore = u.select_atoms('segid I and resid 239 242 246 249 253 256 260 and name CA')

In [30]:
from MDAnalysis.core import AtomGroup
from MDAnalysis.analysis.distances import distance_array
prime_notations = ['-2', '2', '6', '9', '13', '16', '20']
distance_arrays = {}
ag_arrays = {}
for prime_res, at1, at2, at3, at4, at5 in zip(
    prime_notations, ch_1_pore, ch_2_pore, ch_3_pore, ch_4_pore, ch_5_pore):
    print(f'Pore resids: prime {prime_res}, resname {at1.resname} {at2.resname} and resid {at1.resid} {at2.resid} {at3.resid} {at4.resid} {at5.resid}')
    ag_prime = AtomGroup([at1, at2, at3, at4, at5])
    ag_arrays[prime_res] = ag_prime
    distance_arrays[prime_res] = distance_array(ag_prime.positions, ag_prime.positions, box=u.dimensions)

Pore resids: prime -2, resname GLU GLU and resid 247 239 239 247 239
Pore resids: prime 2, resname THR THR and resid 250 242 242 250 242
Pore resids: prime 6, resname SER SER and resid 254 246 246 254 246
Pore resids: prime 9, resname LEU LEU and resid 257 249 249 257 249
Pore resids: prime 13, resname VAL VAL and resid 261 253 253 261 253
Pore resids: prime 16, resname LEU LEU and resid 264 256 256 264 256
Pore resids: prime 20, resname GLU LYS and resid 268 260 260 268 260


In [31]:
def write_pore_restraints_file(fname, prime_res, dist_mat, ag_arr):
    ag_arr_shift = np.roll(ag_arr, -2, axis=0)
    res_groups = [[at1.index+1, at2.index+1] for at1, at2 in zip(ag_arr, ag_arr_shift)]
    dist_groups = [dist_mat[i, j] for i,j in zip([0,1,2,3,4], [2,3,4,0,1])]
    with open(fname, 'w') as f:
        f.write(f'; Pore restraints for prime {prime_res}\n')
        f.write('[ bonds ]\n')
        f.write(';   i     j funct         lo        up1        up2    kdr\n')
        for res_group, dist in zip(res_groups, dist_groups):
            print(res_group, dist)
            f.write(f'{res_group[0]} {res_group[1]} 10 {(dist / 10):6f} {(dist / 10 +0.2):6f} {(dist / 10 +1.2):6f} 5000\n')


for prime_res in prime_notations:
    write_pore_restraints_file(f'../toppar/pore_res_{prime_res}.itp',
                                prime_res,
                                distance_arrays[prime_res],
                                ag_arrays[prime_res])

[3941, 15877] 14.417612446284041
[10012, 21814] 13.68285930304289
[15877, 27885] 14.853463889695416
[21814, 3941] 13.897530429958394
[27885, 10012] 14.158267829245434
[3997, 15931] 11.859845807021705
[10066, 21870] 13.022925146879176
[15931, 27939] 12.226866518319591
[21870, 3997] 12.676234408042497
[27939, 10066] 12.623523037119043
[4060, 15994] 14.39318560340731
[10129, 21933] 14.994614430178157
[15994, 28002] 14.981622949646717
[21933, 4060] 14.574426513195595
[28002, 10129] 14.63960732104219
[4106, 16040] 15.907346241426584
[10175, 21979] 16.463094416944976
[16040, 28048] 16.56375601262031
[21979, 4106] 16.41236372899089
[28048, 10175] 16.595941180373224
[4169, 16102] 16.398612245443445
[10237, 22042] 17.110261601300913
[16102, 28110] 16.54205457009662
[22042, 4169] 16.802008382209042
[28110, 10237] 16.943830424179446
[4224, 16157] 20.894151000048044
[10292, 22097] 20.817307728835612
[16157, 28165] 20.845858628603885
[22097, 4224] 20.684286098853544
[28165, 10292] 20.55444932827970

In [15]:
# Add pore restraints to topol.top
with open('../topol.top', '+a') as f:
    f.write('\n')
    f.write('#ifdef REST_PORE_ON\n')
    f.write('[ intermolecular_interactions]\n')
    for prime_res in prime_notations:
        f.write(f'#include "toppar/pore_res_{prime_res}.itp"\n')
    f.write('#endif\n')